In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os

# Get the current research's directory
research_dir = os.path.dirname(os.path.abspath('__file__'))

# Move one directory back
parent_dir = os.path.dirname(research_dir)

# Change the current working directory to the parent directory
os.chdir(parent_dir)

# Print the current working directory to confirm
print(f"Current working directory: {os.getcwd()}")

In [ ]:
from services.llm import LLMServices

# Inicializa lo necesario (leer YAML, configurar modelos, etc.)
LLMServices.launch()

# Accede directamente al modelo configurado
model = LLMServices.model
embeddings = LLMServices.embeddings


In [ ]:
from langchain_chroma import Chroma

# Define Vector DB
vectorstore = Chroma(
    collection_name="pokemon_series",
    embedding_function=embeddings,
    persist_directory="./.chroma_db",  # Where to save data locally, remove if not necessary
)

In [ ]:
from frank.utils.rag.unstructured import MultiVectorDocumentIndexing

# Index phase
indexing = MultiVectorDocumentIndexing(llm=model, llm_multimodal=model, vectorstore=vectorstore)

indexing.load_pdf('artifacts/rag_docs/EP003 - Ash Catches a Pokémon.pdf')
indexing.split_pdf()
indexing.summarize_elements()
indexing.embed_store_documents()
retriever = indexing.get_retriever()

In [ ]:
indexing.summaries

In [ ]:
docs = retriever.invoke("In chapter 3 of the pokemon series, which pokemon was staring at the moon at night?")

# docs = retriever.invoke("Who is sleeping while Caterpy and Pikachu get to know each other?")

In [ ]:
from frank.utils.rag.processing import parse_docs, show_base64_image

docs_dict = parse_docs(docs)
show_base64_image(docs_dict['images'][0])

In [ ]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.messages import HumanMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser


def build_prompt(kwargs):
    docs_by_type = kwargs["context"]
    user_question = kwargs["question"]

    context_text = ""
    if len(docs_by_type["texts"]) > 0:
        for text_element in docs_by_type["texts"]:
            context_text += text_element.text

    # construct prompt with context (including images)
    prompt_template = f"""
    Answer the question based only on the following context, which can include text, tables, and the below image.
    Context: {context_text}
    Question: {user_question}
    """

    prompt_content = [{"type": "text", "text": prompt_template}]

    if len(docs_by_type["images"]) > 0:
        for image in docs_by_type["images"]:
            prompt_content.append(
                {
                    "type": "image_url",
                    "image_url": {"url": f"data:image/jpeg;base64,{image}"},
                }
            )

    return ChatPromptTemplate.from_messages(
        [
            HumanMessage(content=prompt_content),
        ]
    )


chain = (
    {
        "context": retriever | RunnableLambda(parse_docs),
        "question": RunnablePassthrough(),
    }
    | RunnableLambda(build_prompt)
    | model
    | StrOutputParser()
)

chain_with_sources = {
    "context": retriever | RunnableLambda(parse_docs),
    "question": RunnablePassthrough(),
} | RunnablePassthrough().assign(
    response=(
        RunnableLambda(build_prompt)
        | model
        | StrOutputParser()
    )
)

In [ ]:
response = chain.invoke(
    "In the chapter 3 of pokemon series, which is the pokemon who was staring at the moon in the night?"
)
print(response)